In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# remover comentarios si sed quiere instalar la libreria 
# publicada en pipy
#%pip uninstall azure-datalake-utils -y
#%pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple azure-datalake-utils==0.2.0.dev23
#%pip show azure-datalake-utils

In [3]:
import sys

import pandas as pd

sys.path.append('../')

In [4]:
import azure.functions as func 
from mod_test import *
from centraal_dataframework.blueprints import framework

app = func.FunctionApp() 
app.register_functions(framework) 


In [5]:
# Call the function.
a = app.get_functions()


In [6]:
trigger = a[1].get_user_function()

In [ ]:
from azure.functions import QueueMessage
msg= QueueMessage(body = bytes("hello",  "utf-8"))
trigger(msg)